In [19]:

import os
from pathlib import Path
from typing import Any, Dict, List, Tuple

import h5py
import pandas as pd
from google.cloud import storage
from google.oauth2 import service_account

# Config (override via .env)
BUCKET_NAME = os.getenv("GCP_BUCKET_NAME", "inkling-ssvep-emg")
BASE_PREFIX = "EMG-nature/data/"
LOCAL_ROOT = Path("./data_emg_nature")
SERVICE_ACCOUNT_KEY = os.getenv(
    "GOOGLE_APPLICATION_CREDENTIALS",
    "./keys/inkling-479911-b140632c44c5.json",
)


In [20]:


# def create_gcs_client(key_path: str = SERVICE_ACCOUNT_KEY) -> storage.Client:
#     """Authenticate to GCS using a local service account key."""
#     os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
#     creds = service_account.Credentials.from_service_account_file(key_path)
#     return storage.Client(credentials=creds)


# def list_participant_folders(client: storage.Client, bucket_name: str = BUCKET_NAME,
#                              base_prefix: str = BASE_PREFIX) -> List[str]:
#     """Find participant prefixes under the dataset path (fallbacks included)."""
#     iterator = client.list_blobs(bucket_name, prefix=base_prefix, delimiter="/")
#     prefixes = set()
#     for page in iterator.pages:
#         prefixes.update(page.prefixes)
#     participant_prefixes = sorted(
#         p for p in prefixes if p.rstrip("/").split("/")[-1].startswith("participant_")
#     )
#     if participant_prefixes:
#         return participant_prefixes

#     candidates = set()
#     for blob in client.list_blobs(bucket_name, prefix=base_prefix):
#         parts = blob.name.split("/")
#         for idx, part in enumerate(parts):
#             if part.startswith("participant_"):
#                 cand = "/".join(parts[: idx + 1]) + "/"
#                 candidates.add(cand)
#     return sorted(candidates)


# def download_participant_files(client: storage.Client, participant_prefix: str,
#                                bucket_name: str = BUCKET_NAME,
#                                local_root: Path = LOCAL_ROOT) -> Tuple[str, str]:
#     """Download required files for one participant into LOCAL_ROOT."""
#     bucket = client.bucket(bucket_name)
#     participant_id = participant_prefix.rstrip("/").split("/")[-1]
#     dest_dir = local_root / participant_id
#     dest_dir.mkdir(parents=True, exist_ok=True)

#     required = ["emg_data.hdf5", "glove_data.hdf5", "finger_data.hdf5", "trials.csv"]
#     for fname in required:
#         blob_path = f"{participant_prefix}{fname}"
#         blob = bucket.blob(blob_path)
#         target = dest_dir / fname
#         if target.exists():
#             continue
#         if not blob.exists():
#             print(f"⚠ Missing in bucket: {blob_path}")
#             continue
#         try:
#             blob.download_to_filename(target)
#         except Exception as exc:
#             print(f"⚠ Failed to download {blob_path}: {exc}")
#     return str(dest_dir), participant_id


# def read_first_dataset(path: Path):
#     """Load the first dataset in an HDF5 file; return None if not present."""
#     if not path.exists():
#         return None
#     with h5py.File(path, "r") as h5f:
#         keys = list(h5f.keys())
#         if not keys:
#             return None
#         return h5f[keys[0]][()]


# def load_participant_data(participant_id: str, participant_dir: str) -> pd.DataFrame:
#     """Load trials and raw arrays for a single participant into a DataFrame."""
#     participant_dir = Path(participant_dir)
#     trials_path = participant_dir / "trials.csv"
#     trials = pd.read_csv(trials_path) if trials_path.exists() else pd.DataFrame()

#     emg = read_first_dataset(participant_dir / "emg_data.hdf5")
#     glove = read_first_dataset(participant_dir / "glove_data.hdf5")
#     finger = read_first_dataset(participant_dir / "finger_data.hdf5")

#     lengths = []
#     if not trials.empty:
#         lengths.append(len(trials))
#     for arr in (emg, glove, finger):
#         if arr is not None and hasattr(arr, "shape"):
#             lengths.append(arr.shape[0])
#     if not lengths:
#         return pd.DataFrame()

#     n = min(lengths)
#     if trials.empty:
#         trials = pd.DataFrame({"trial_id": range(n)})

#     rows: List[Dict[str, Any]] = []
#     for i in range(n):
#         meta = trials.iloc[i].to_dict()
#         rows.append({
#             **meta,
#             "participant_id": participant_id,
#             "emg_raw": emg[i] if emg is not None else None,
#             "glove_raw": glove[i] if glove is not None else None,
#             "finger_raw": finger[i] if finger is not None else None,
#         })
#     return pd.DataFrame(rows)


# def build_master_dataframe(participants: List[Tuple[str, str]]) -> pd.DataFrame:
#     """Aggregate all participants into one DataFrame."""
#     frames = [load_participant_data(pid, pdir) for pdir, pid in participants]
#     return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()


In [21]:

# # Authenticate, download, and load the full dataset
# client = create_gcs_client()
# participant_prefixes = list_participant_folders(client)
# print(f"Participants found in bucket: {len(participant_prefixes)}")
# if not participant_prefixes:
#     print("No participant prefixes detected; check bucket contents and BASE_PREFIX.")

# LOCAL_ROOT.mkdir(parents=True, exist_ok=True)
# results: List[Tuple[str, str]] = []
# for prefix in participant_prefixes:
#     results.append(download_participant_files(client, prefix))

# emg_df = build_master_dataframe(results)

# print(f"emg_df.shape: {emg_df.shape}")
# print("emg_df.head():")
# print(emg_df.head())
# if not emg_df.empty:
#     sample = emg_df.loc[0, "emg_raw"]
#     if hasattr(sample, "shape"):
#         print("First trial EMG shape:", sample.shape)
# else:
#     print("No data loaded; verify downloads in", LOCAL_ROOT)


In [22]:
def create_gcs_client(key_path: str = SERVICE_ACCOUNT_KEY) -> storage.Client:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
    creds = service_account.Credentials.from_service_account_file(key_path)
    return storage.Client(credentials=creds)


In [23]:
def list_participant_folders(client, bucket_name=BUCKET_NAME, base_prefix=BASE_PREFIX):
    """Return all participant_X/ prefixes under EMG-nature/data/."""
    prefixes = set()
    for blob in client.list_blobs(bucket_name, prefix=base_prefix):
        parts = blob.name.split("/")
        for i, part in enumerate(parts):
            if part.startswith("participant_"):
                prefixes.add("/".join(parts[:i+1]) + "/")
    return sorted(prefixes)


In [24]:
def list_block_folders(client, participant_prefix, bucket_name=BUCKET_NAME):
    """Return all participant_dayX_blockY/ prefixes."""
    blocks = set()
    for blob in client.list_blobs(bucket_name, prefix=participant_prefix):
        parts = blob.name.split("/")
        for i, part in enumerate(parts):
            if "block" in part:
                blocks.add("/".join(parts[:i+1]) + "/")
    return sorted(blocks)


In [25]:
def download_block_files(client, block_prefix, participant_id, bucket_name=BUCKET_NAME, local_root=LOCAL_ROOT):
    bucket = client.bucket(bucket_name)
    block_id = block_prefix.rstrip("/").split("/")[-1]

    dest_dir = local_root / participant_id / block_id
    dest_dir.mkdir(parents=True, exist_ok=True)

    for blob in client.list_blobs(bucket_name, prefix=block_prefix):
        if blob.name.endswith("/"):
            continue

        target = dest_dir / Path(blob.name).name
        if not target.exists():
            print(f"Downloading {blob.name} → {target}")
            blob.download_to_filename(target)

    return dest_dir, block_id


In [26]:
def read_first_dataset(path: Path):
    if not path.exists():
        return None
    with h5py.File(path, "r") as h5f:
        keys = list(h5f.keys())
        if not keys:
            return None
        return h5f[keys[0]][()]


In [27]:
def load_block(block_dir: Path, participant_id: str, block_id: str) -> pd.DataFrame:
    trials_path = block_dir / "trials.csv"
    trials = pd.read_csv(trials_path) if trials_path.exists() else pd.DataFrame()

    emg = read_first_dataset(block_dir / "emg_data.hdf5")
    glove = read_first_dataset(block_dir / "glove_data.hdf5")
    finger = read_first_dataset(block_dir / "finger_data.hdf5")

    lengths = []
    if not trials.empty:
        lengths.append(len(trials))
    for arr in (emg, glove, finger):
        if arr is not None:
            lengths.append(arr.shape[0])

    if not lengths:
        return pd.DataFrame()

    n = min(lengths)

    if trials.empty:
        trials = pd.DataFrame({"trial_id": range(n)})

    rows = []
    for i in range(n):
        row = trials.iloc[i].to_dict()
        row.update({
            "participant_id": participant_id,
            "block_id": block_id,
            "emg_raw": emg[i] if emg is not None else None,
            "glove_raw": glove[i] if glove is not None else None,
            "finger_raw": finger[i] if finger is not None else None,
        })
        rows.append(row)

    return pd.DataFrame(rows)


In [28]:
def build_master_dataframe(client):
    participant_prefixes = list_participant_folders(client)

    all_frames = []

    for pfx in participant_prefixes:
        participant_id = pfx.rstrip("/").split("/")[-1]
        print(f"🔍 Participant: {participant_id}")

        blocks = list_block_folders(client, pfx)
        for block_pfx in blocks:
            print(f"   📦 Block: {block_pfx}")

            block_dir, block_id = download_block_files(client, block_pfx, participant_id)
            df = load_block(block_dir, participant_id, block_id)
            if not df.empty:
                all_frames.append(df)

    return pd.concat(all_frames, ignore_index=True)


In [29]:
client = create_gcs_client()

LOCAL_ROOT.mkdir(exist_ok=True)

emg_df = build_master_dataframe(client)

print(emg_df.shape)
print(emg_df.head())


🔍 Participant: participant_1
   📦 Block: EMG-nature/data/participant_1/participant1_day1_block1/
   📦 Block: EMG-nature/data/participant_1/participant1_day1_block2/
   📦 Block: EMG-nature/data/participant_1/participant1_day2_block1/
   📦 Block: EMG-nature/data/participant_1/participant1_day2_block2/
🔍 Participant: participant_2
   📦 Block: EMG-nature/data/participant_2/participant2_day1_block1/
   📦 Block: EMG-nature/data/participant_2/participant2_day1_block2/
   📦 Block: EMG-nature/data/participant_2/participant2_day2_block1/
   📦 Block: EMG-nature/data/participant_2/participant2_day2_block2/
🔍 Participant: participant_3
   📦 Block: EMG-nature/data/participant_3/participant3_day1_block1/
   📦 Block: EMG-nature/data/participant_3/participant3_day1_block2/
   📦 Block: EMG-nature/data/participant_3/participant3_day2_block1/
   📦 Block: EMG-nature/data/participant_3/participant3_day2_block2/
🔍 Participant: participant_4
   📦 Block: EMG-nature/data/participant_4/participant4_day1_block1/


In [30]:
emg_df.head()

,Unnamed: 0,row_number,target_position,grasp,trial_no,block,participant_id,block_id,emg_raw,glove_raw,finger_raw
0,0,0,2,3,0,0,participant_1,participant1_day1_block1,"[3.763498e-05, 1.9842508e-05, 9.071698e-06, 1....","[86.6102294921875, 86.6102294921875, 86.610229...","[16.80420684814453, 16.80420684814453, 16.8042..."
1,1,1,2,3,1,0,participant_1,participant1_day1_block1,"[2.4820081e-05, 2.6200492e-05, 2.5797071e-05, ...","[0.8576509952545166, 0.8576509952545166, 0.857...","[1.467085599899292, 1.467085599899292, 1.46708..."
2,2,2,2,3,2,0,participant_1,participant1_day1_block1,"[7.703583e-06, 9.893e-06, 1.0767478e-05, 9.083...","[2.8023266792297363, 2.8023266792297363, 2.802...","[4.928889751434326, 4.928889751434326, 4.92888..."
3,3,3,2,3,3,0,participant_1,participant1_day1_block1,"[8.394901e-06, 7.721386e-06, 7.572158e-06, 6.6...","[-55.870609283447266, -55.870609283447266, -55...","[6.696066856384277, 6.696066856384277, 6.69606..."
4,4,4,2,3,4,0,participant_1,participant1_day1_block1,"[-1.6433607e-05, -1.6947637e-05, -2.4275832e-0...","[90.43270111083984, 90.43270111083984, 90.4327...","[-0.38769450783729553, -0.38769450783729553, -..."
